<a href="https://colab.research.google.com/github/aditya161205/NLP/blob/main/RAG_LangHug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -q langchain transformers sentence-transformers faiss-cpu pypdf accelerate langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import files
import os

print("Please upload your PDF document...")
uploaded = files.upload()

if uploaded:
    file_name = next(iter(uploaded))
    print(f"\nUploaded file '{file_name}' successfully.")
else:
    print("No file was uploaded.")
    file_name = None

Please upload your PDF document...


Saving QuantArc_SOP (1) (1).pdf to QuantArc_SOP (1) (1).pdf

Uploaded file 'QuantArc_SOP (1) (1).pdf' successfully.


In [29]:
pip install -q langchain-community

In [38]:
import torch
from langchain_community.document_loaders import PyPDFLoader # Updated import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.chains import RetrievalQA


# --- 1. Load and Split Document ---
print("Step 1: Loading and splitting the document...")
loader = PyPDFLoader(file_name)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)
print(f"-> Document split into {len(docs)} chunks.")

# --- 2. Create Embeddings and Vector Store (on GPU) ---
print("\nStep 2: Creating embeddings and vector store...")
# Use a Hugging Face model for embeddings, running on the GPU
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'} # Use 'cuda' for GPU
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Create the vector store using FAISS
vectorstore = FAISS.from_documents(docs, embeddings)
print("-> Vector store created successfully.")

# --- 3. Set Up the LLM (on GPU) ---
print("\nStep 3: Setting up the LLM...")
# Use a flan-t5 model from Hugging Face
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id) # The model will automatically be placed on the GPU

# Create a text-generation pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device=0 # device=0 for GPU on Colab
)
llm = HuggingFacePipeline(pipeline=pipe)
print("-> LLM pipeline ready.")

# --- 4. Create and Run the RAG Chain ---
print("\nStep 4: Creating the RAG chain...")
retriever = vectorstore.as_retriever()
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
print("-> RAG chain created. Ready to answer questions.")

Step 1: Loading and splitting the document...
-> Document split into 11 chunks.

Step 2: Creating embeddings and vector store...
-> Vector store created successfully.

Step 3: Setting up the LLM...


Device set to use cuda:0


-> LLM pipeline ready.

Step 4: Creating the RAG chain...
-> RAG chain created. Ready to answer questions.


In [37]:

question = "Who is Khushi tongiya?"
print(f"'{question}'")
result = chain.run(question)

print("\n Answer:")
print(result)


Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors


'Who is Khushi tongiya?'

 Answer:
Industry Expert
